In [1]:
import time

import meshcat_shapes
import numpy as np
from numpy.linalg import norm, solve

import pinocchio
from robot_descriptions.loaders.pinocchio import load_robot_description

def solve_ik():
    robot = load_robot_description("ur3_description")
    model = robot.model
    data = robot.data

    viz = pinocchio.visualize.MeshcatVisualizer(
        robot.model, robot.collision_model, robot.visual_model
    )
    robot.setVisualizer(viz, init=False)
    viz.initViewer(open=True)
    viz.loadViewerModel()

    viewer = viz.viewer
    meshcat_shapes.frame(viewer["end_effector_target"], opacity=0.5)
    meshcat_shapes.frame(viewer["end_effector"], opacity=1.0)
    JOINT_ID = 6
    oMdes = pinocchio.SE3(np.eye(3), np.array([0.01, 0.1, 0.3]))  # target location of end effector
    viewer["end_effector_target"].set_transform(oMdes.np)

    q = pinocchio.neutral(model)
    eps = 1e-6
    IT_MAX = 100000
    DT = 1e-1
    damp = 1e-12
    i = 0
    while True:
        pinocchio.forwardKinematics(model, data, q)
        iMd = data.oMi[JOINT_ID].actInv(oMdes)

        err = pinocchio.log(iMd).vector  # in joint frame
        if norm(err) < eps:
            success = True
            break
        if i >= IT_MAX:
            success = False
            break
        J = pinocchio.computeJointJacobian(model, data, q, JOINT_ID)
        J = pinocchio.Jlog6(iMd.inverse()) @ J
        v = J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
        q = pinocchio.integrate(model, q, v * DT)
        viewer["end_effector"].set_transform(data.oMi[JOINT_ID].np)
        viz.display(q)
        if not i % 10:
            print("%d: error = %s" % (i, err.T))
        i += 1
        time.sleep(DT)

    if success:
        print("Convergence achieved!")
    else:
        print(
            "\nWarning: the iterative algorithm has not reached convergence "
            "to the desired precision"
        )






You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


0: error = [-0.3667024  -0.01235    -0.70198888 -0.         -3.14159265 -0.        ]
10: error = [-1.09640137e-01 -5.06063818e-03 -7.85687059e-03 -9.14632636e-04
 -1.09540535e+00 -1.43804025e-03]
20: error = [-3.84128751e-02 -2.90406116e-03 -3.01487753e-03 -1.46917707e-04
 -3.81962759e-01 -4.64750260e-04]
30: error = [-1.78296821e-02 -1.17649607e-02  4.05976669e-03  1.11354661e-05
 -1.33211910e-01 -1.51032302e-04]
40: error = [ 1.14934650e-01  1.15938607e-01  2.73874445e-01  4.83259471e-05
 -4.64180510e-02 -5.10267515e-05]
50: error = [ 5.21809698e-03 -1.30670794e-02  6.68381174e-02  1.11277832e-05
 -1.63057905e-02 -1.78368545e-05]
60: error = [-1.78555413e-03 -1.22044942e-02  2.89497985e-02  1.70871915e-03
 -6.25644950e-03  1.27633855e-05]
70: error = [ 0.0318639  -0.00989178 -0.11168339  0.07887965 -0.00517829  0.10888153]
80: error = [ 0.04421703 -0.01870666  0.00264416 -0.00657917 -0.00656497  0.03327717]
90: error = [ 0.01371235 -0.01221974  0.01396687 -0.00325998 -0.00298534  0.0

KeyboardInterrupt: 

In [8]:
print("\nresult: %s" % q.flatten().tolist())
print("\nfinal error: %s" % err.T)


result: [-0.4129330932353817, -34.2341294229671, 8.374558148679295, 31.677593105425565, -6.691173974862958, -2.6812580199330043]

final error: [-0.004134   -0.01147027  0.0054074  -0.01767866  0.00933737  0.00072506]
